In [1]:
import requests 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
request = requests.get(url).text

bs = BeautifulSoup(request, "xml")
table = bs.find("table")
columns = ["Postalcode","Borough","Neighborhood"]
df = pd.DataFrame(columns = columns)
for tr_cell in table.find_all("tr"):
    row_data = []
    for td_cell in tr_cell.find_all("td"):
        row_data.append(td_cell.text.strip())
    if len(row_data) == 3:
        df.loc[len(df)] = row_data
        
#drop if Borough is not assigned
df = df[df["Borough"] != "Not assigned"] 
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
#merge neighborhoods in the same postal code area
df_joined = df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df_joined = df_joined.reset_index(drop = False)
df_joined.rename(columns = {'Neighborhood':'Neighborhood_joined'}, inplace = True)

df1 = pd.merge(df, df_joined, on = 'Postalcode')
df1.drop(['Neighborhood'],axis = 1,inplace = True)
df1.drop_duplicates(inplace = True)
df1.rename(columns = {'Neighborhood_joined':'Neighborhood'},inplace=True)
df1.head(10)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
df1.shape

(103, 3)